## Adding LL97 Fines

In [1]:
import pandas as pd
import numpy as np

def current_carbon_emission(row):
    cce_electricity = row['Electricity Use - Grid Purchase (kWh)'] * 0.000288962
    cce_natural_gas = row['Natural Gas Use (kBtu)'] * 0.00005311
    cce_fuel_2 = row['Fuel Oil #2 Use (kBtu)'] * 0.00007421
    cce_fuel_4 = row['Fuel Oil #4 Use (kBtu)'] * 0.00007529
    cce_district_steam = row['District Steam Use (kBtu)'] * 0.00004493

    if np.isnan(row['Electricity Use - Grid Purchase (kWh)']):
        cce_electricity = 0
    if np.isnan(row['Natural Gas Use (kBtu)']):
        cce_natural_gas = 0
    if np.isnan(row['Fuel Oil #2 Use (kBtu)']):
        cce_fuel_2 = 0
    if np.isnan(row['Fuel Oil #4 Use (kBtu)']):
        cce_fuel_4 = 0
    if np.isnan(row['District Steam Use (kBtu)']):
        cce_district_steam = 0

    cce_total = cce_electricity + cce_natural_gas + cce_fuel_2 + cce_fuel_4 + cce_district_steam
    return cce_total

def finding_factor_2024(property_type):
    factor_2024 = df_mapping[df_mapping['ll84_property_type']==property_type]['factor_2024'].values[0]
    return factor_2024

def finding_factor_2030(property_type):
    factor_2030 = df_mapping[df_mapping['ll84_property_type']==property_type]['factor_2030'].values[0]
    return factor_2030

def carbon_cap_2024(row):
    p_type_1 = row['Largest Property Use Type']
    p_type_2 = row['2nd Largest Property Use Type']
    p_type_3 = row['3rd Largest Property Use Type']
    factor_2024_1 = finding_factor_2024(p_type_1)
    factor_2024_2 = finding_factor_2024(p_type_2)
    factor_2024_3 = finding_factor_2024(p_type_3)
    if np.isnan(row['Largest Property Use Type - Gross Floor Area (ft²)']):
        sqft_1 = 0
    else: 
        sqft_1 = row['Largest Property Use Type - Gross Floor Area (ft²)']
    if np.isnan(row['2nd Largest Property Use - Gross Floor Area (ft²)']):
        sqft_2 = 0
    else: 
        sqft_2 = row['2nd Largest Property Use - Gross Floor Area (ft²)']
    if np.isnan(row['3rd Largest Property Use Type - Gross Floor Area (ft²)']):
        sqft_3 = 0
    else: 
        sqft_3 = row['3rd Largest Property Use Type - Gross Floor Area (ft²)']
    carbon_cap_2024_1  = factor_2024_1*sqft_1
    carbon_cap_2024_2  = factor_2024_2*sqft_2
    carbon_cap_2024_3  = factor_2024_3*sqft_3
    total_carbon_cap_2024 = carbon_cap_2024_1+carbon_cap_2024_2+carbon_cap_2024_3
    return total_carbon_cap_2024

def carbon_cap_2030(row):
    p_type_1 = row['Largest Property Use Type']
    p_type_2 = row['2nd Largest Property Use Type']
    p_type_3 = row['3rd Largest Property Use Type']
    factor_2030_1 = finding_factor_2030(p_type_1)
    factor_2030_2 = finding_factor_2030(p_type_2)
    factor_2030_3 = finding_factor_2030(p_type_3)
    if np.isnan(row['Largest Property Use Type - Gross Floor Area (ft²)']):
        sqft_1 = 0
    else: 
        sqft_1 = row['Largest Property Use Type - Gross Floor Area (ft²)']
    if np.isnan(row['2nd Largest Property Use - Gross Floor Area (ft²)']):
        sqft_2 = 0
    else: 
        sqft_2 = row['2nd Largest Property Use - Gross Floor Area (ft²)']
    if np.isnan(row['3rd Largest Property Use Type - Gross Floor Area (ft²)']):
        sqft_3 = 0
    else: 
        sqft_3 = row['3rd Largest Property Use Type - Gross Floor Area (ft²)']
    carbon_cap_2030_1  = factor_2030_1*sqft_1
    carbon_cap_2030_2  = factor_2030_2*sqft_2
    carbon_cap_2030_3  = factor_2030_3*sqft_3
    total_carbon_cap_2030 = carbon_cap_2030_1+carbon_cap_2030_2+carbon_cap_2030_3
    return total_carbon_cap_2030

def calculate_fines_2024(row):
    current_carbon_emission = row['carbon_emission_2020']
    carbon_cap_2024 = row['LL97_carbon_cap_2024']
    if current_carbon_emission < carbon_cap_2024:
        fines_2024 = 0
    else:
        fines_2024 = (current_carbon_emission-carbon_cap_2024)*268
    return fines_2024

def calculate_fines_2030(row):
    current_carbon_emission = row['carbon_emission_2020']
    carbon_cap_2030 = row['LL97_carbon_cap_2030']
    if current_carbon_emission < carbon_cap_2030:
        fines_2030 = 0
    else:
        fines_2030 = (current_carbon_emission-carbon_cap_2030)*268
    return fines_2030

In [3]:
df = pd.read_excel("Submission Data - PRC Management (1).xlsx", skiprows=0)

# mapping LL84 property type with LL97 property type
df_mapping = pd.read_csv('data/property_type_mapping.csv')

df_mapping['factor_2024'] = pd.to_numeric(df_mapping['factor_2024'], errors='coerce')
df_mapping['factor_2030'] = pd.to_numeric(df_mapping['factor_2030'], errors='coerce')
df['Largest Property Use Type - Gross Floor Area (ft²)'] = pd.to_numeric(df['Largest Property Use Type - Gross Floor Area (ft²)'], errors='coerce')
df['2nd Largest Property Use - Gross Floor Area (ft²)'] = pd.to_numeric(df['2nd Largest Property Use - Gross Floor Area (ft²)'], errors='coerce')
df['3rd Largest Property Use Type - Gross Floor Area (ft²)'] = pd.to_numeric(df['3rd Largest Property Use Type - Gross Floor Area (ft²)'], errors='coerce')
df['Total GHG Emissions (Metric Tons CO2e)'] = pd.to_numeric(df['Total GHG Emissions (Metric Tons CO2e)'], errors='coerce')

df['Electricity Use - Grid Purchase (kWh)'] = pd.to_numeric(df['Electricity Use - Grid Purchase (kWh)'], errors='coerce')
df['Natural Gas Use (kBtu)'] = pd.to_numeric(df['Natural Gas Use (kBtu)'], errors='coerce')
df['Fuel Oil #2 Use (kBtu)'] = pd.to_numeric(df['Fuel Oil #2 Use (kBtu)'], errors='coerce')
df['Fuel Oil #4 Use (kBtu)'] = pd.to_numeric(df['Fuel Oil #4 Use (kBtu)'], errors='coerce')
df['District Steam Use (kBtu)'] = pd.to_numeric(df['District Steam Use (kBtu)'], errors='coerce')

df['carbon_emission_2020'] = df.apply(current_carbon_emission, axis=1) # changed column name from "current_carbon_emission" -> "carbon_emissions_2020"
df['LL97_carbon_cap_2024'] = df.apply(carbon_cap_2024, axis=1) # changed column name from "carbon_cap_2024" -> "LL97_carbon_cap_2024"
df['LL97_carbon_cap_2030'] = df.apply(carbon_cap_2030, axis=1) # changed column name from "carbon_cap_2030" -> "LL97_carbon_cap_2030"
df['LL97_fines_2024'] = df.apply(calculate_fines_2024, axis=1) # changed column name from "fines_2024" -> "LL97_fines_2024"
df['LL97_fines_2030'] = df.apply(calculate_fines_2030, axis=1) # changed column name from "fines_2030" -> "LL97_fines_2030"

df.to_excel('Submission Data - PRC Management (1)_complete.xlsx')

In [ ]:
# df = pd.read_csv('/Users/jaeseongpark/Desktop/brightpower/mining_ll84/data/nyc_ll84/Energy_and_Water_Data_Disclosure_for_Local_Law_84_2021__Data_for_Calendar_Year_2020_.csv')

# # convert 'Property ID' datatype from 'int64' to 'string'
# df['Property Id'] = df['Property Id'].astype('string')

# # drop child property rows (1806 = 28067-26261)
# filter1 = df['Parent Property Id']!='Not Applicable: Standalone Property'
# filter2 = df['Property Id']!=df['Parent Property Id']
# drop_index_list = df[filter1&filter2].index
# df.drop(labels=drop_index_list, inplace=True)

# # add number of property types column
# def get_num_of_property_types(property_types_str):
#     property_types_list = property_types_str.split(", ")
#     return (len(property_types_list))
# df['num_of_property_types'] = df['List of All Property Use Types at Property'].apply(get_num_of_property_types) # there are 1275 (=26261-24986) rows with more than 3 property types #len(df) - sum(df['num_of_property_types'].value_counts()[:3])

# # drop rows with more than 3 property types (1275 = 26261-24986)
# filter3 = df['num_of_property_types']>3
# drop_index_list2 = df[filter3].index
# df.drop(labels=drop_index_list2, inplace=True)

# ## Calculate 'Total Gross Floor Area (ft²)'
# # cols_list = df.columns.to_list()
# # target_cols = [i for i in cols_list if 'Gross Floor Area' in i]
# # for col in target_cols:
# #     df[col] = pd.to_numeric(df[col], errors='coerce')
# # df['Total Gross Floor Area (ft²)'] = df[target_cols].apply(lambda x: x.sum(), axis=1)

# # creat property type set
# ll84_property_types = set(df['Largest Property Use Type'].unique().tolist())
# for i in df['2nd Largest Property Use Type'].unique().tolist():
#     ll84_property_types.add(i)
# for i in df['3rd Largest Property Use Type'].unique().tolist():
#     ll84_property_types.add(i)
# ll84_property_types = list(ll84_property_types)

# # mapping LL84 property type with LL97 property type
# df_mapping = pd.read_csv('/Users/jaeseongpark/Desktop/brightpower/mining_ll84/report/add_ll97_fines/data/property_type_mapping.csv')

# df_mapping['factor_2024'] = pd.to_numeric(df_mapping['factor_2024'], errors='coerce')
# df_mapping['factor_2030'] = pd.to_numeric(df_mapping['factor_2030'], errors='coerce')
# df['Largest Property Use Type - Gross Floor Area (ft²)'] = pd.to_numeric(df['Largest Property Use Type - Gross Floor Area (ft²)'], errors='coerce')
# df['2nd Largest Property Use - Gross Floor Area (ft²)'] = pd.to_numeric(df['2nd Largest Property Use - Gross Floor Area (ft²)'], errors='coerce')
# df['3rd Largest Property Use Type - Gross Floor Area (ft²)'] = pd.to_numeric(df['3rd Largest Property Use Type - Gross Floor Area (ft²)'], errors='coerce')
# df['Total GHG Emissions (Metric Tons CO2e)'] = pd.to_numeric(df['Total GHG Emissions (Metric Tons CO2e)'], errors='coerce')

# def finding_factor_2024(property_type):
#     factor_2024 = df_mapping[df_mapping['ll84_property_type']==property_type]['factor_2024'].values[0]
#     return factor_2024

# def finding_factor_2030(property_type):
#     factor_2030 = df_mapping[df_mapping['ll84_property_type']==property_type]['factor_2030'].values[0]
#     return factor_2030

# def carbon_cap_2024_largest(row):
#     p_type_1 = row['Largest Property Use Type']
#     p_type_2 = row['2nd Largest Property Use Type']
#     p_type_3 = row['3rd Largest Property Use Type']
#     factor_2024_1 = finding_factor_2024(p_type_1)
#     factor_2024_2 = finding_factor_2024(p_type_2)
#     factor_2024_3 = finding_factor_2024(p_type_3)
#     if np.isnan(row['Largest Property Use Type - Gross Floor Area (ft²)']):
#         sqft_1 = 0
#     else: 
#         sqft_1 = row['Largest Property Use Type - Gross Floor Area (ft²)']
#     if np.isnan(row['2nd Largest Property Use - Gross Floor Area (ft²)']):
#         sqft_2 = 0
#     else: 
#         sqft_2 = row['2nd Largest Property Use - Gross Floor Area (ft²)']
#     if np.isnan(row['3rd Largest Property Use Type - Gross Floor Area (ft²)']):
#         sqft_3 = 0
#     else: 
#         sqft_3 = row['3rd Largest Property Use Type - Gross Floor Area (ft²)']
#     carbon_cap_2024_1  = factor_2024_1*sqft_1
#     carbon_cap_2024_2  = factor_2024_2*sqft_2
#     carbon_cap_2024_3  = factor_2024_3*sqft_3
#     total_carbon_cap_2024 = carbon_cap_2024_1+carbon_cap_2024_2+carbon_cap_2024_3
#     return total_carbon_cap_2024

# def carbon_cap_2030_largest(row):
#     p_type_1 = row['Largest Property Use Type']
#     p_type_2 = row['2nd Largest Property Use Type']
#     p_type_3 = row['3rd Largest Property Use Type']
#     factor_2030_1 = finding_factor_2030(p_type_1)
#     factor_2030_2 = finding_factor_2030(p_type_2)
#     factor_2030_3 = finding_factor_2030(p_type_3)
#     if np.isnan(row['Largest Property Use Type - Gross Floor Area (ft²)']):
#         sqft_1 = 0
#     else: 
#         sqft_1 = row['Largest Property Use Type - Gross Floor Area (ft²)']
#     if np.isnan(row['2nd Largest Property Use - Gross Floor Area (ft²)']):
#         sqft_2 = 0
#     else: 
#         sqft_2 = row['2nd Largest Property Use - Gross Floor Area (ft²)']
#     if np.isnan(row['3rd Largest Property Use Type - Gross Floor Area (ft²)']):
#         sqft_3 = 0
#     else: 
#         sqft_3 = row['3rd Largest Property Use Type - Gross Floor Area (ft²)']
#     carbon_cap_2030_1  = factor_2030_1*sqft_1
#     carbon_cap_2030_2  = factor_2030_2*sqft_2
#     carbon_cap_2030_3  = factor_2030_3*sqft_3
#     total_carbon_cap_2030 = carbon_cap_2030_1+carbon_cap_2030_2+carbon_cap_2030_3
#     return total_carbon_cap_2030

# def calculate_fines_2024(row):
#     current_carbon_emission = row['Total GHG Emissions (Metric Tons CO2e)']
#     carbon_cap_2024 = row['carbon_cap_2024']
#     if current_carbon_emission < carbon_cap_2024:
#         fines_2024 = 0
#     else:
#         fines_2024 = (current_carbon_emission-carbon_cap_2024)*268
#     return fines_2024

# def calculate_fines_2030(row):
#     current_carbon_emission = row['Total GHG Emissions (Metric Tons CO2e)']
#     carbon_cap_2030 = row['carbon_cap_2030']
#     if current_carbon_emission < carbon_cap_2030:
#         fines_2030 = 0
#     else:
#         fines_2030 = (current_carbon_emission-carbon_cap_2030)*268
#     return fines_2030

# def floor_area_sum(row):
#     if np.isnan(row['Largest Property Use Type - Gross Floor Area (ft²)']):
#         sqft_1 = 0
#     else: 
#         sqft_1 = row['Largest Property Use Type - Gross Floor Area (ft²)']
#     if np.isnan(row['2nd Largest Property Use - Gross Floor Area (ft²)']):
#         sqft_2 = 0
#     else: 
#         sqft_2 = row['2nd Largest Property Use - Gross Floor Area (ft²)']
#     if np.isnan(row['3rd Largest Property Use Type - Gross Floor Area (ft²)']):
#         sqft_3 = 0
#     else: 
#         sqft_3 = row['3rd Largest Property Use Type - Gross Floor Area (ft²)']
#     total = sqft_1 + sqft_2 + sqft_3
#     return total

# df['floor_area_sum'] = df.apply(floor_area_sum, axis=1)
# df['carbon_cap_2024'] = df.apply(carbon_cap_2024_largest, axis=1)
# df['carbon_cap_2030'] = df.apply(carbon_cap_2030_largest, axis=1)
# df['fines_2024'] = df.apply(calculate_fines_2024, axis=1)
# df['fines_2030'] = df.apply(calculate_fines_2030, axis=1)

# df.to_csv('df_ll84_2020_final.csv')